Create a model to predict WR yards vs upcoming team

In [89]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage
from PIL import Image
import numpy as np
from io import BytesIO
import requests
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [90]:
# import data to get schedule and player stats
nfl_sched = nfl.import_schedules([2023])
weekly_nfl = nfl.import_weekly_data([2023])

Downcasting floats.


In [91]:
# gather WR stats
wr_df_two = weekly_nfl[(weekly_nfl["position"] == "WR")]
new_columns = ['player_display_name','recent_team', 'opponent_team','fantasy_points', 'fantasy_points_ppr','receiving_yards','receiving_yards_after_catch','receptions','targets','receiving_tds','receiving_air_yards','receiving_first_downs','receiving_epa','target_share','air_yards_share','racr','wopr']
wr_df = wr_df_two[new_columns]
wr_df= wr_df[wr_df['fantasy_points_ppr'] != 0]
wr_df.head()
    

,player_display_name,recent_team,opponent_team,fantasy_points,fantasy_points_ppr,receiving_yards,receiving_yards_after_catch,receptions,targets,receiving_tds,receiving_air_yards,receiving_first_downs,receiving_epa,target_share,air_yards_share,racr,wopr
16,Randall Cobb,NYJ,NE,1.2,2.2,12.0,10.0,1,2,0,6.0,1.0,0.441822,0.071429,0.037500,2.000000,0.133393
17,Randall Cobb,NYJ,KC,0.8,2.8,8.0,1.0,2,4,0,24.0,1.0,-1.630082,0.102564,0.082474,0.333333,0.211578
19,Randall Cobb,NYJ,PHI,2.0,2.0,0.0,0.0,0,3,0,22.0,0.0,-1.435370,0.090909,0.084942,0.000000,0.195823
31,Marvin Jones,DET,KC,-1.2,0.8,8.0,4.0,2,6,0,50.0,0.0,-6.195231,0.187500,0.230415,0.160000,0.442540
33,Marvin Jones,DET,CAR,2.2,4.2,22.0,5.0,2,2,0,17.0,1.0,0.981102,0.074074,0.102410,1.294118,0.182798


In [92]:
#create df for average player stats vs opponent
wr_mean_df = wr_df.groupby('player_display_name').mean().reset_index()
wr_mean_df.set_index('player_display_name', inplace=True)
wr_mean_df.head()

,fantasy_points,fantasy_points_ppr,receiving_yards,receiving_yards_after_catch,receptions,targets,receiving_tds,receiving_air_yards,receiving_first_downs,receiving_epa,target_share,air_yards_share,racr,wopr
player_display_name,,,,,,,,,,,,,,
A.J. Brown,13.200000,20.200001,112.000000,39.0,7.000000,10.000000,0.333333,140.500000,5.333333,6.352998,0.305502,0.459329,0.786219,0.779783
Adam Thielen,12.916667,21.083334,84.833336,29.5,8.166667,9.833333,0.666667,76.833336,5.500000,6.694321,0.266896,0.313504,1.175475,0.619796
Alec Pierce,2.483333,4.316667,24.833334,3.5,1.833333,3.666667,0.000000,60.166668,1.500000,1.159167,0.098360,0.210584,0.538673,0.294948
Allen Lazard,4.500000,6.833333,35.000000,6.0,2.333333,3.833333,0.166667,51.000000,2.000000,2.132671,0.147079,0.284828,0.732628,0.419998
Allen Robinson,2.740000,6.140000,27.400000,6.8,3.400000,5.200000,0.000000,33.599998,0.800000,-0.216133,0.158754,0.127071,0.874933,0.327081


In [93]:
wr_mean_df.fillna(0, inplace=True)  # Fill NaN values with zeros in the DataFrame 'df'


In [94]:
# create ml to group into classes
# OVR MODEL
def get_clusters(k, data):
    # Create a copy of the DataFrame
    data = data.copy()

    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Fit the model
    model.fit(data)

    # Predict clusters
    predictions = model.predict(data)

    # Create return DataFrame with predicted clusters
    data["class"] = model.labels_

    return data

In [95]:
# get classes
# create ml 5 ks

classes_wr = get_clusters(5, wr_mean_df)
classes_wr.head()

wr_average = classes_wr.reset_index(drop=False)
wr_average.head()

,player_display_name,fantasy_points,fantasy_points_ppr,receiving_yards,receiving_yards_after_catch,receptions,targets,receiving_tds,receiving_air_yards,receiving_first_downs,receiving_epa,target_share,air_yards_share,racr,wopr,class
0,A.J. Brown,13.200000,20.200001,112.000000,39.0,7.000000,10.000000,0.333333,140.500000,5.333333,6.352998,0.305502,0.459329,0.786219,0.779783,1
1,Adam Thielen,12.916667,21.083334,84.833336,29.5,8.166667,9.833333,0.666667,76.833336,5.500000,6.694321,0.266896,0.313504,1.175475,0.619796,3
2,Alec Pierce,2.483333,4.316667,24.833334,3.5,1.833333,3.666667,0.000000,60.166668,1.500000,1.159167,0.098360,0.210584,0.538673,0.294948,0
3,Allen Lazard,4.500000,6.833333,35.000000,6.0,2.333333,3.833333,0.166667,51.000000,2.000000,2.132671,0.147079,0.284828,0.732628,0.419998,0
4,Allen Robinson,2.740000,6.140000,27.400000,6.8,3.400000,5.200000,0.000000,33.599998,0.800000,-0.216133,0.158754,0.127071,0.874933,0.327081,2


In [96]:
# create df for defense vs player
wr_stats_team = wr_df.groupby(['opponent_team', 'player_display_name']).sum().reset_index()
wr_stats_team.head()

,opponent_team,player_display_name,fantasy_points,fantasy_points_ppr,receiving_yards,receiving_yards_after_catch,receptions,targets,receiving_tds,receiving_air_yards,receiving_first_downs,receiving_epa,target_share,air_yards_share,racr,wopr
0,ARI,Andrei Iosivas,0.900000,1.900000,9.0,0.0,1,2,0,25.0,0.0,-0.727112,0.045455,0.083893,0.360000,0.126907
1,ARI,Brandin Cooks,1.700000,3.700000,17.0,3.0,2,7,0,52.0,2.0,-5.286209,0.189189,0.275132,0.326923,0.476376
2,ARI,Brandon Aiyuk,14.800000,20.799999,148.0,19.0,6,6,0,129.0,6.0,10.005992,0.285714,0.712707,1.147287,0.927466
3,ARI,CeeDee Lamb,6.200000,10.200000,53.0,15.0,4,7,0,52.0,2.0,-1.518677,0.189189,0.275132,1.019231,0.476376
4,ARI,Cooper Kupp,20.799999,27.799999,148.0,51.0,7,9,1,113.0,5.0,10.231497,0.428571,0.624309,1.309734,1.079874


In [97]:
#add _against to all the columns in yards vs team
wr_vs_stats = wr_stats_team.columns = [col + '_vs' for col in wr_stats_team.columns]
wr_vs_stats

['opponent_team_vs',
 'player_display_name_vs',
 'fantasy_points_vs',
 'fantasy_points_ppr_vs',
 'receiving_yards_vs',
 'receiving_yards_after_catch_vs',
 'receptions_vs',
 'targets_vs',
 'receiving_tds_vs',
 'receiving_air_yards_vs',
 'receiving_first_downs_vs',
 'receiving_epa_vs',
 'target_share_vs',
 'air_yards_share_vs',
 'racr_vs',
 'wopr_vs']

In [98]:
wr_stats_team.colummns = wr_vs_stats
wr_stats_team.rename(columns={'opponent_team_vs_vs': 'opponent_team', 'player_display_number': 'player_display_name'}, inplace=True)

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [99]:
wr_stats_team['player_display_name'] = wr_stats_team['player_display_name_vs']
wr_stats_team = wr_stats_team.drop(columns=['player_display_name_vs'])

wr_stats_team.head()


,opponent_team_vs,fantasy_points_vs,fantasy_points_ppr_vs,receiving_yards_vs,receiving_yards_after_catch_vs,receptions_vs,targets_vs,receiving_tds_vs,receiving_air_yards_vs,receiving_first_downs_vs,receiving_epa_vs,target_share_vs,air_yards_share_vs,racr_vs,wopr_vs,player_display_name
0,ARI,0.900000,1.900000,9.0,0.0,1,2,0,25.0,0.0,-0.727112,0.045455,0.083893,0.360000,0.126907,Andrei Iosivas
1,ARI,1.700000,3.700000,17.0,3.0,2,7,0,52.0,2.0,-5.286209,0.189189,0.275132,0.326923,0.476376,Brandin Cooks
2,ARI,14.800000,20.799999,148.0,19.0,6,6,0,129.0,6.0,10.005992,0.285714,0.712707,1.147287,0.927466,Brandon Aiyuk
3,ARI,6.200000,10.200000,53.0,15.0,4,7,0,52.0,2.0,-1.518677,0.189189,0.275132,1.019231,0.476376,CeeDee Lamb
4,ARI,20.799999,27.799999,148.0,51.0,7,9,1,113.0,5.0,10.231497,0.428571,0.624309,1.309734,1.079874,Cooper Kupp


In [100]:
wr_test_data = pd.merge(wr_stats_team, wr_average, on='player_display_name', how='inner')
wr_test_data.dtypes







opponent_team_vs                   object
fantasy_points_vs                 float32
fantasy_points_ppr_vs             float32
receiving_yards_vs                float32
receiving_yards_after_catch_vs    float32
receptions_vs                       int32
targets_vs                          int32
receiving_tds_vs                    int32
receiving_air_yards_vs            float32
receiving_first_downs_vs          float32
receiving_epa_vs                  float32
target_share_vs                   float32
air_yards_share_vs                float32
racr_vs                           float32
wopr_vs                           float32
player_display_name                object
fantasy_points                    float32
fantasy_points_ppr                float32
receiving_yards                   float32
receiving_yards_after_catch       float32
receptions                        float64
targets                           float64
receiving_tds                     float64
receiving_air_yards               

In [101]:
column_names = [
    'fantasy_points_vs',
    'fantasy_points_ppr_vs',
    'receiving_yards_after_catch_vs',
    'targets_vs',
    'receiving_air_yards_vs',
    'receiving_first_downs_vs',
    'receiving_epa_vs',
    'target_share_vs',
    'air_yards_share_vs',
    'racr_vs',
    'wopr_vs',
    'fantasy_points',
    'fantasy_points_ppr',
    'receiving_yards',
    'receiving_yards_after_catch',
    'receptions',
    'targets',
    'receiving_tds',
    'receiving_air_yards',
    'receiving_first_downs',
    'receiving_epa',
    'target_share',
    'air_yards_share',
    'racr',
    'wopr'
    
]

In [102]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Split data into features (X) and target variable (y)
X = wr_test_data[column_names]
y = wr_test_data[['receptions_vs', 'receiving_yards_vs', 'receiving_tds_vs']]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train_scaled, y_train)

# Make predictions
predictions = model.predict(X_test_scaled)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")


Mean Absolute Error (MAE): 2.352650600198684
Mean Squared Error (MSE): 29.999426998034377
Root Mean Squared Error (RMSE): 3.2269847812926855
R-squared (R2): 0.9432432070767316


In [103]:
# create averages for a team to use in trained model
team_averages = wr_stats_team[wr_stats_team['opponent_team_vs'] == 'KC']
team_averages.mean()

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


fantasy_points_vs                  5.333001
fantasy_points_ppr_vs              8.933001
receiving_yards_vs                38.250000
receiving_yards_after_catch_vs    11.150000
receptions_vs                      3.600000
targets_vs                         6.100000
receiving_tds_vs                   0.250000
receiving_air_yards_vs            58.599998
receiving_first_downs_vs           2.050000
receiving_epa_vs                   0.225633
target_share_vs                    0.182300
air_yards_share_vs                 0.226733
racr_vs                            0.659416
wopr_vs                            0.432163
dtype: float64

In [104]:
#get average stats for a player
player_averages = wr_average[wr_average['player_display_name'] == 'Jerry Jeudy']
player_averages.mean()


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


fantasy_points                  4.440000
fantasy_points_ppr              8.440001
receiving_yards                44.400002
receiving_yards_after_catch    14.400000
receptions                      4.000000
targets                         5.800000
receiving_tds                   0.000000
receiving_air_yards            70.199997
receiving_first_downs           2.200000
receiving_epa                   0.613817
target_share                    0.211694
air_yards_share                 0.355940
racr                            0.606766
wopr                            0.566699
class                           0.000000
dtype: float64

In [105]:
# Create the test data 
data_dict = {
    'fantasy_points_vs': 5.258890,
    'fantasy_points_ppr_vs': 8.870000,
    'receiving_yards_after_catch_vs': 11.722222,
    'targets_vs': 6.166667,
    'receiving_air_yards_vs': 60.388889,
    'receiving_first_downs_vs':  2.111111,
    'receiving_epa_vs': 0.376688,
    'target_share_vs': 0.174778,
    'air_yards_share_vs': 0.211564,
    'racr_vs':  0.659351,
    'wopr_vs': 0.410262,
    'fantasy_points':  5.2,
    'fantasy_points_ppr': 9.45,
    'receiving_yards': 52,
    'receiving_yards_after_catch': 15.75,
    'receptions': 4.250000,
    'targets': 6.200000,
    'receiving_tds':  0.100000,
    'receiving_air_yards': 79,
    'receiving_first_downs':  2.500000,
    'receiving_epa': 2.27,
    'target_share': 0.21,
    'air_yards_share': 0.37,
    'racr': 0.66,
    'wopr':  0.57
    
}


In [106]:
5.20
9.45
52.00
15.75
4.25
6.00
0.00
79.00
2.50
2.27
0.21
0.37
0.66
0.57

0.57

In [107]:
# Create a sample untrained dataset for X using the average values
data_untrained_test = {feature: [data_dict[feature] for _ in range(4)] for feature in data_dict}
X_untrained = pd.DataFrame(data_untrained_test)

In [108]:
# 2. Use the trained model to make predictions on X_untrained.
predictions_untrained_team = model.predict(X_untrained)
print(predictions_untrained_team)

[[ 3.95865168e+01 -1.38400036e+06 -3.59686715e+05]
 [ 3.95865168e+01 -1.38400036e+06 -3.59686715e+05]
 [ 3.95865168e+01 -1.38400036e+06 -3.59686715e+05]
 [ 3.95865168e+01 -1.38400036e+06 -3.59686715e+05]]


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/base.py:439: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
